### PTUSE logger

This notebook produces csv files which can be imported into a google spreadsheet.

Searches for all ptuse observations in the archive

#### Import python routines

In [ ]:
import pysolr
from IPython.core.display import display, HTML

import katdal
import katpoint

import os
import glob

import csv
import json

archive = pysolr.Solr('http://192.168.1.50:8983/solr/kat_core')

### Setup variables

Puls='All' to search for all pulsars or Puls='JNAME' to search for a particular pulsar

In [ ]:
Puls='All'

# set up the instruction set
instr_str=[]
instr_str.append('InstructionSet:beamform_single_pulsar*')
#instr_str.append('InstructionSet:beamform_*cal*')
instr_string = '(' +' OR '.join(instr_str) + ')'

startdate = '2018-01-01T00:00:00Z'
enddate = '2018-12-01T00:00:00Z'

outputfilenm="ptuse."+Puls+".Log.csv"

In [ ]:
def get_delays(filename):
    
    data = katdal.open(filename)
    ref=data.ref_ant
    utc=data.start_time
    ants=data.ants
    nants=len(ants)
    antenna_names = [ants[i].name for i in range(nants)]
    print antenna_names
    antnames=[]
    for ant in data.ants:
        antnames.append(ant.name)
        if ant.name==data.ref_ant:
            HDel=ant.delay_model.params['FIX_H'].value
            VDel=ant.delay_model.params['FIX_V'].value
    ants=" ".join(antnames)
#    HDel=data.ref_ant.delay_model.params['FIX_H'].value
#    VDel=data.ref_ant.delay_model.params['FIX_V'].value


    NAnts=len(antnames)
    array_ant = katpoint.Antenna('ref', *(data.ants[0].ref_position_wgs84))
    #print array_ant,NAnts
    correlator_model = katpoint.DelayCorrection(data.ants, array_ant)
    delay=correlator_model.max_delay
    
    return delay,utc,NAnts,ants,ref,HDel,VDel

## Get links to observation reports

In [ ]:
def get_obsreport_link(scheduleID):

    ar_res = archive.search('CAS.ProductTypeName:MeerKATAR1ReductionProduct AND ExperimentID:%s' % scheduleID, sort="StartTime desc")    
    if ar_res.hits > 0:
        redfiles = ar_res.docs[0]["CAS.ReferenceDatastore"] 

        for fl in redfiles:
            if fl.endswith('html') or fl.endswith('pdf'):
               
                link=fl[5:]
                obsrep=link.replace('/var/kat','http://kat-archive.kat.ac.za')
    else:

        obsrep="None"
    
    return obsrep

# Search archive

In [ ]:
search_string=[]

search_str = []
search_str.append(instr_string)

search_str.append('CAS.ProductTypeName:MeerKATAR1TelescopeProduct')
if (Puls != "All"):
    search_str.append('Targets:"%s"'%Puls)

search_str.append('StartTime:["%s" TO "%s"]'%(startdate,enddate))
search_string = ' AND '.join(search_str)

print "Searching archive for %s" %search_str
ar_res = archive.search( search_string ,rows=10000,**{'sort':'StartTime asc'})
Observations=ar_res.docs
print "Found %d hits" %ar_res.hits

ObsCount = ar_res.hits

In [ ]:
outputfile=open(outputfilenm,"w")
print "Writing observation log to %s" %(outputfilenm)

print "\n\n"

outputfile.write("ScheduleBlock;PTQA;name;target;Observer;StartTime;duration;NAnts;AntNames;progress_report;archivelink;")
outputfile.write("VisFile;ptuseData;phaseup_sb;phaseup_target;phaseupTime;phaseupQA1;thisQA;phaseup_h5;phaseup_obsreport;phaseup_archive;phaseup_progress;instruction_set\n")


Count = 0
for r in Observations:
    Count = Count + 1
    
    SB = r['ScheduleBlockIdCode']
    print "Processing %s: %d of %d" %(SB,Count,ObsCount)
    try:
        Observer = r['Observer']
        target = r['Targets'][0]
        
    except:
        Observer="None"
        target="None"
        print "no meta data"
        #continue
    duration=r['Duration']
    Start=r['StartTime']
    
    ants=' '.join(r['Antennas'])
    NAnts=len(r['Antennas'])
    
    if (NAnts < 4):
        continue
    VisFile=r['CAS.ReferenceDatastore'][0][5:]
    progress='http://obs.mkat.karoo.kat.ac.za:8081/tailtask/%s/progress'%(SB)
    archivelink = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"a":true,"b":true,"ob":false,"rs":{}}'%(SB)
    instructions = r['InstructionSet']
    QA = 3
    phasesb='None'
    phaseobsrep='None'
    phasesb='None'
    phasesrc='None'
    
    phaseres = archive.search('CAS.ProductTypeName:MeerKATAR1TelescopeProduct AND InstructionSet:bf_phaseup* AND StartTime:[* TO %s]' % (r['StartTime']), sort="StartTime desc")

    if phaseres.hits > 0:
            try:
                QA = phaseres.docs[0]['QA1']
            except:
                QA = 3
            
            phasesb = phaseres.docs[0]['ScheduleBlockIdCode']
            ins = phaseres.docs[0]['InstructionSet']
            #print ins
            flatten = 0
            #print ins.find("flatten")
            if (ins.find("flatten")>0):
                print "using flatten"
                flatten = 1
            try:
                phasesrc=  phaseres.docs[0]['Targets'][0]
                pharchivelink = 'http://kat-archive.kat.ac.za:8080/archive_search/#{"ei":"%s","so":true,"a":true,"b":true,"ob":false,"rs":{}}'%(phasesb)

                phaseprogress='http://obs.mkat.karoo.kat.ac.za:8081/tailtask/%s/progress'%(phasesb)
            
                PhaseUpTime=phaseres.docs[0]['StartTime']
                phaseupfile=phaseres.docs[0]['CAS.ReferenceDatastore'][0][5:]
                
                phaseobsrep=get_obsreport_link(phasesb)
            except:
                phaseobsrep="None"
                phasesrc = "None"



    
    search_str = []
    search_str.append('ScheduleBlockIdCode:"%s"'%SB)

    search_str.append('CAS.ProductTypeName:PTUSETimingArchiveProduct')

    search_string = ' AND '.join(search_str)
    
    sb_res = archive.search( search_string ,rows=10000)
    

    ptuseData="None"
    #print "PTUSE search %d" %(sb_res.hits)
    if sb_res.hits > 0:
        
        for s in sb_res.docs:
            if s['CAS.ProductTypeName']=='PTUSETimingArchiveProduct':
                duration=s['Duration']
                try:
                    PTQA = s['QA1']
                except:
                    PTQA = 3
                
                ptuseData=s['CAS.ReferenceDatastore'][0][5:]
                data_files = os.listdir(ptuseData)
                firstfile=sorted(data_files)[0]
                utc=firstfile.replace('-','',2).replace(':','',2).replace('-','T').replace('.ar','')
                name=target+'.'+utc
                ptuseFileSize=sum(s['CAS.ReferenceFileSize'])/1024.0**3
    if (QA < 0):
        print "\t\t\t Phaseup failed"
    if (ptuseData=="None"):
        print "\t\t\t No ptuse data"
        name=str(Start)
    
    outputfile.write("%s;%d;%s;%s;%s;%s;" %(SB,PTQA,name,target,Observer,Start))
                                      
    outputfile.write("%d;%d;%s;%s;" %(duration,NAnts,ants,progress))
    outputfile.write("%s;%s;%s;%s;" %(archivelink,VisFile,ptuseData,phasesb))
    outputfile.write("%s;%s;%s; ;" %(phasesrc,PhaseUpTime,QA))
    outputfile.write("%s;%s;%s;%s;%d;%s\n" %(phaseupfile,phaseobsrep,pharchivelink,phaseprogress,flatten,instructions))
    
    
outputfile.close()  
        